Table of Content
=============

* [1. Introduction](#introduction)
* [2. Key Points](#key-points)
* [3. Implementation](#implementation)
  * [3.1. Import Statements](#import-statements)
  * [3.2. Core Functions](#core-functions)
  * [3.3. Network Example](#network-example)
  * [3.4. Iterative Algorithm](#iterative-algorithm)
  * [3.5. Final Solution](#final-solution)

# 1. Introduction <a class='anchor' id='introduction'></a>

This Notebook provides an implementation of the work of J.B. Ward and H.W. Wale, titled *Digital Computer Solution of Power-Flow Problems*, from June 1956. 

The full original paper can be found [here](http://helios.acomp.usf.edu/~fehr/wardhale.pdf).

# 2. Key Points <a class='anchor' id='key-points'></a>
    
* This is the first paper to propose a method for solving power flow (or load flow) problems on a digital computer.
* This solution method is often referred to in the literature a


iterative method is based on the [Gauss-Seidel numerical method](https://en.wikipedia.org/wiki/Gauss%E2%80%93Seidel_method) and is often referred to as the 
* The proposed iterative method is composed of 5 steps:
    1. Describe the network by its admittance matrix $Y = G + jB$.
    2. Find an iterative solution for terminal (bus) voltages, which satisfies the specified fixed variables. These fixed variables are: (a) voltage magnitude and angle at the slack bus, (b) active and reactive power injections at PQ (load) buses, and (c) active power injection and voltage magnitude at PV (generator) buses.
    3. Compute the final terminal (bus) variables: active and reactive power injections, complex voltage, and current injections.
    4. Compute the total net loss as the sum of all bus active power injections.
    5. Optionally (if required), individual line flows can be computed.

# 3. Implementation <a class='anchor' id='implementation'></a>

## 3.1. Import Statements <a class='anchor' id='import-statements'></a>

In [12]:
import numpy as np

## 3.2. Core Functions <a class='anchor' id='core-functions'></a>

In [13]:
def compute_I(G, B, E_real, E_imag, k):
    """ 
    Compute the nodal I injection at bus k (eqns. 4-6). 
    
    Parameters
    ----------
    G : np.ndarray
        The (N, N) nodal conductance matrix, such that G = Re(Y).
    B : np.ndarray
        The (N, N) nodal susceptance matrix, such that B = Im(Y).
    E_real : np.ndarray
        The (N,) vector of the real component of the nodal voltages.
    E_imag : np.ndarray
        The (N,) vector of the imaginary component of the nodal voltages.
    k : int
        The index of the node at which to compute the curent injection I_k
        (0-indexed).
        
    Returns
    -------
    i_real : float
        The real part of the nodal current injection at bus k, Re(I_k).
    i_imag : float
        The imaginary part of the nodal current injection at bus k, Im(I_k).
    """
    
    n = Y.shape[0]
    i_real, i_imag = 0, 0
    for m in range(Y.shape[0]):
        i_real += G[k, m] * E_real[m] - B[k, m] * E_imag[m]
        i_imag += G[k, m] * E_imag[m] + B[k, m] * E_real[m]
        
    return i_real, i_imag

In [14]:
def compute_PQ(i_real, i_imag, e_real, e_imag):
    """ 
    Compute the nodal power injection S = P + jQ at a bus k (eqns. 7-8).
    
    Parameters
    ----------
    i_real : float
        The real part of the current injection at bus k, Re(I_k).
    i_imag : float
        The imaginary part of the current injection at bus k, Im(I_k).
    e_real : float
        The real part of the voltage at bus k, Re(E_k).
    e_imag : float
        The imaginary part of the voltage at bus k, Im(E_k).
    
    Returns
    -------
    p : float
        The active power injection at bus k, P_k.
    q : float
        The reactive power injection at bus k, Q_k.
    """
    
    p = i_real * e_real + i_imag * e_imag
    q = i_real * e_imag - i_imag * e_real

    return p, q

In [15]:
def compute_E_correction_load(p_specified, q_specified, i_real, i_imag,
                              e_real, e_imag, B, G, k):
    """ 
    Compute the corrective voltage at load terminals (eqns. 9-13).
    
    Parameters:
    ----------
    p_specified : float
        The scheduled active power injection at bus k.
    q_specified : float
        The scheduled reactive power injection at bus k.
    i_real : float
        The real part of the current injection at bus k, Re(I_k).
    i_imag : float
        The imaginary part of the current injection at bus k, Im(I_k).
    e_real : float
        The real part of the voltage at bus k, Re(E_k).
    e_imag : float
        The imaginary part of the voltage at bus k, Im(E_k).
    B : np.ndarray
        The (N, N) nodal susceptance matrix, such that B = Im(Y).
    G : np.ndarray
        The (N, N) nodal conductance matrix, such that G = Re(Y).
    k : int
        The index of the node (0-indexed).
        
    Returns
    -------
    x0 : float
        The real part of the corrective voltage at bus k.
    x1 : float
        The imaginary part of the corrective voltage at bus k.
    """

    # Compute deviation of P, Q from specified values.
    p, q = compute_PQ(i_real, i_imag, e_real, e_imag)
    dp = p_specified - p
    dq = q_specified - q
    
    # Solve the system of linear equations.
    A_00 = e_real * G[k, k] + e_imag * B[k, k] + i_real
    A_01 = - e_real * B[k, k] + e_imag * G[k, k] + i_imag
    A_10 = - e_real * B[k, k] + e_imag * G[k, k] - i_imag
    A_11 = - e_real * G[k, k] + e_imag * B[k, k] + i_real
    A = np.array([[A_00, A_01], [A_10, A_11]])
    b = np.array([[dp], [dq]])
    
    # Correction x is [e_real, e_imag].
    x = np.dot(np.linalg.inv(A), b)
    
    return x[0], x[1]

In [16]:
def compute_E_correction_generator(p_specified, e_magn_specified, i_real, i_imag,
                                   e_real, e_imag, B, G, k):
    """ 
    Compute the corrective voltage at generator terminals (eqns. 14-20).
    
    Parameters:
    ----------
    p_specified : float
        The scheduled active power injection at bus k.
    e_magn_specified : float
        The specified voltage magnitude at bus k.
    i_real : float
        The real part of the current injection at bus k, Re(I_k).
    i_imag : float
        The imaginary part of the current injection at bus k, Im(I_k).
    e_real : float
        The real part of the voltage at bus k, Re(E_k).
    e_imag : float
        The imaginary part of the voltage at bus k, Im(E_k).
    B : np.ndarray
        The (N, N) nodal susceptance matrix, such that B = Im(Y).
    G : np.ndarray
        The (N, N) nodal conductance matrix, such that G = Re(Y).
    k : int
        The index of the node (0-indexed).
        
    Returns
    -------
    x0 : float
        The real part of the corrective voltage at bus k.
    x1 : float
        The imaginary part of the corrective voltage at bus k.
    """
    
    # Compute deviation of P, |E|^2 from specified values.
    p, _ = compute_PQ(i_real, i_imag, e_real, e_imag)
    dp = p_specified - p
    e_magn_square = e_real ** 2 + e_imag ** 2
    de_square = e_magn_specified ** 2 - e_magn_square
    
    # Solve the system of linear equations.
    A_00 = e_real * G[k, k] + e_imag * B[k, k] + i_real
    A_01 = - e_real * B[k, k] + e_imag * G[k, k] + i_imag
    A_10 = 2 * e_real
    A_11 = 2 * e_imag
    A = np.array([[A_00, A_01], [A_10, A_11]])
    b = np.array([[dp], [de_square]])
    
    # Correction x is [e_real, e_imag].
    x = np.dot(np.linalg.inv(A), b)
    
    return x[0], x[1]

## 3.3. Network Example <a class='anchor' id='network_example'></a>

In [17]:
# Number of nodes.
n = 6

In [18]:
# Conductance matrix.
G = np.diag([0.992203, 1.021401, 0.444860, 
             1.112371, 0.576541, 0.988036])
G[1, 2] = G[2, 1] = - G[2, 2]
G[0, 3] = G[3, 0] = - 0.558269
G[1, 4] = G[4, 1] = - 0.576541
G[0, 5] = G[5, 0] = - 0.433934
G[3, 5] = G[5, 3] = - 0.554102

# Susceptance matrix.
B = np.diag([-4.375561, -1.954525, -8.164860, 
             -13.975358, -4.64170, -7.619402])
B[1, 2] = B[2, 1] = 0.646063
B[0, 3] = B[3, 0] = 2.581096
B[2, 3] = B[3, 2] = 8.270677
B[1, 4] = B[4, 1] = 1.308462
B[0, 5] = B[5, 0] = 1.827463
B[3, 5] = B[5, 3] = 2.324944
B[4, 5] = B[5, 4] = 3.416666

# Admittance matrix Y = G + jB.
Y = G + 1j * B
print('Y matrix is symmetric: ', np.array_equal(Y, Y.T))

Y matrix is symmetric:  True


In [19]:
# Specified terminal conditions (per unit).
Es_magn = np.array([1.05, 1.10, None, None, None, None])
Es_ang = np.array([0, None, None, None, None, None])
Ps = np.array([None, 0.5, -0.55, 0, -0.30, -0.50])
Qs = np.array([None, None, -0.13, 0, -0.18, -0.05])

## 3.4. Iterative algorithm <a class='anchor' id='iterative-algorithm'></a>

In [20]:
# Number of iterations to perform.
n_iters = 60

In [21]:
# Step 1: start with a flat guess for E (except for known value at slack bus.)
slack_E = Es_magn[0] * np.exp(Es_ang[0])
E_real = np.array([slack_E.real] + [1.] * (n-1))
E_imag = np.array([slack_E.imag] + [0.] * (n-1))

In [22]:
# Iterate for `iters` iterations.
for i in range(n_iters):
    
    # Loop over all non-slack buses.
    for k in range(1, n):
        
        # Step 2: compute the current injection at bus k.
        i_real, i_imag = compute_I(G, B, E_real, E_imag, k)

        # Step 3a: compute the E correction to be made.        
        
        # Case a: k is a load bus.
        if Ps[k] is not None and Qs[k] is not None: 
            de_real, de_imag = compute_E_correction_load(Ps[k], Qs[k], i_real, i_imag, 
                                                         E_real[k], E_imag[k], B, G, k)
        
        # Case b: k is a generator bus.
        elif Ps[k] is not None and Es_magn[k] is not None:  # Case b: k is a generator bus.
            de_real, de_imag = compute_E_correction_generator(Ps[k], Es_magn[k], i_real, i_imag, 
                                                              E_real[k], E_imag[k], B, G, k)
            
        else:
            raise NotImplementedError()
            
        # Step 3b: update the corrected voltage at bus k.
        E_real[k] += de_real
        E_imag[k] += de_imag

## 3.5. Final solution <a class='anchor' id='final-solution'></a>

In [23]:
P, Q, I_real, I_imag = [], [], [], []

for k in range(n):

    # Compute the current injection at each bus.
    i_real, i_imag = compute_I(G, B, E_real, E_imag, k)
    I_real.append(i_real)
    I_imag.append(i_imag)
    
    # Compute the active and reactive power injections at each bus.
    p, q = compute_PQ(i_real, i_imag, E_real[k], E_imag[k])
    P.append(p)
    Q.append(q)
        
E = E_real + 1j * E_imag
E_magn = np.abs(E)
E_ang = np.angle(E)
        
# Print the final solution.
np.set_printoptions(precision=5, suppress=True)
print('Final solution:\n')
print('E: ', E_magn)
print('Degrees: ', E_ang * 180 / np.pi)
print('P: ', np.array(P))
print('Q: ', np.array(Q))
print('I_real: ', np.array(I_real))
print('I_imag', np.array(I_imag))

print('\nFinal loss = %.5f' % np.sum(P))

Final solution:

E:  [1.05    1.1     1.00032 0.92939 0.91923 0.91898]
Degrees:  [  0.       -3.3651  -12.78945  -9.83854 -12.34126 -12.24384]
P:  [ 0.95229  0.5     -0.55     0.      -0.3     -0.5    ]
Q:  [ 0.43715  0.18571 -0.13     0.      -0.18    -0.05   ]
I_real:  [ 0.90694  0.44385 -0.50741  0.      -0.27697 -0.52016]
I_imag [-0.41633 -0.19522  0.24844 -0.       0.26104  0.16855]

Final loss = 0.10229
